Doc Similarity Model Sistem Rekomendasi Film || Nicholaus Verdhy Putranto || A11.2020.12447

# Penjelasan Projek

Proyek rekomendasi film berdasarkan isi konten adalah sebuah aplikasi atau sistem yang memanfaatkan berbagai elemen informasi film, seperti nama pemain (cast), judul film (title), genre, sinopsis (taline), kata kunci (keyword), dan ringkasan (overview), untuk memberikan rekomendasi film kepada pengguna. Dengan menganalisis dan memahami elemen-elemen ini dari film-film yang telah dilihat atau disukai oleh pengguna, sistem ini dapat merekomendasikan film-film serupa atau yang sesuai dengan preferensi individu pengguna. Informasi ini digunakan untuk menghasilkan rekomendasi yang lebih akurat dan sesuai dengan selera pengguna, membantu mereka menemukan film-film baru yang mereka mungkin akan nikmati berdasarkan berbagai aspek isi konten yang relevan.

## Work Flow

Untuk Workflow pada projek ini diantaranya
1. Data collection : mengumpulkan data, datan data ditampilkan pada jupyternbotebook
2. Data cleansing : Digunakan untuk membersihkan data yang hilang
3. Feature enginering 1 : Melakukan pemilihan fitur, yang nantinya akan membuat fitur baru dengan concating
4. Preprocesing data 2 : Tokenisasi dan stop word yang mengumpulkanhasilkan dokumen yang telah normalisasi
5. Feater Extraction : melakuakn extrasi fitur dengan mengubah data teks jadi angka dengan  TF-IDF
6. Feature engginering 2 : melakukan penghitungan kemiripan dokumen dengan cosine skor
7. Membuat sistem rekomendasi    

# Membuat Model

## Import Library

In [1]:
#untuk data set
import pandas as pd
import numpy as np

#Normalisasi Data
import nltk
import re
import contractions

#feature extraction (mengubah jadi angka)
from sklearn.feature_extraction.text import TfidfVectorizer

#Menghitungang kesamaan skor kata dengan yang dicocokan
from sklearn.metrics.pairwise import cosine_similarity

#untuk pencarian kata typo yang diketikan user yang dicocokan kedalam dataset
import difflib
import sys

#set tampilan
pd.set_option("Display.max_column",None)

#ignore warning
import warnings
warnings.filterwarnings("ignore")

## Data Colection

### Loading Dataset

In [2]:
data_film = pd.read_csv("Dataset/movies.csv")
data_film

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honey

### Seputar Dataset

In [3]:
data_film.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [4]:
data_film.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

## Data Cleansing

In [5]:
data_film.fillna(" ",axis=0,inplace=True)

In [6]:
data_film.isna().sum()

index                   0
budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
cast                    0
crew                    0
director                0
dtype: int64

## Feature Engginering

### Selection Feature ( concating)

In [7]:
data_film["konten_film"] = data_film["genres"]+" "+data_film["overview"]+data_film["keywords"]+" "+data_film["tagline"]+" "+data_film["cast"]+" "+data_film["director"]

In [8]:
data_film["konten_film"].iloc[1]

"Adventure Fantasy Action Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.ocean drug abuse exotic island east india trading company love of one's life At the end of the world, the adventure begins. Johnny Depp Orlando Bloom Keira Knightley Stellan Skarsg\\u00e5rd Chow Yun-fat Gore Verbinski"

## Preprocessing data lanjutan

### Stop Word dan Tokenisasi

In [9]:
#Memangil StopWord
stop_words = nltk.corpus.stopwords.words('english')

#### Membuat fungsi untuk normalisasi dokumen teks

In [10]:
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

#### Normalisasi

In [11]:
normalisasi_corpus = np.vectorize(normalize_document)

hasil_normalize_corpus = normalisasi_corpus(list(data_film['konten_film']))
len(hasil_normalize_corpus)

4803

## Feature Extracton

TFidf Extraction

In [12]:
#memanggil method Tfidf ke dlam vectorizer
vectorizer = TfidfVectorizer()

In [13]:
#konvert
fitur_vectors = vectorizer.fit_transform(hasil_normalize_corpus)

In [14]:
print(fitur_vectors)

  (0, 5179)	0.13082672561043435
  (0, 17341)	0.08292751898763055
  (0, 28271)	0.14755081472409795
  (0, 21703)	0.12095026922175944
  (0, 18966)	0.16870511969841476
  (0, 31504)	0.11410847651935507
  (0, 35761)	0.14830998770909637
  (0, 30221)	0.15645743825466366
  (0, 28779)	0.16541074330396702
  (0, 36936)	0.14830998770909637
  (0, 36475)	0.17960565297971468
  (0, 28827)	0.10875794157123647
  (0, 36391)	0.07534471390118924
  (0, 10924)	0.1434293941180993
  (0, 30733)	0.13363786440637587
  (0, 6754)	0.17053345046996676
  (0, 35552)	0.08618302476541573
  (0, 30960)	0.24397841811729248
  (0, 13170)	0.11103189514171408
  (0, 6373)	0.15250159662784718
  (0, 6337)	0.21671301008857902
  (0, 1265)	0.12363456364937903
  (0, 26367)	0.163915062632112
  (0, 24061)	0.1575541154810948
  (0, 12575)	0.1363319392802255
  :	:
  (4802, 3073)	0.29265387239990437
  (4802, 9959)	0.2854125237692977
  (4802, 4682)	0.10822776922254122
  (4802, 7407)	0.10822776922254122
  (4802, 29368)	0.08328957317433085
  (4

## Feature Enginering lanjutan

Dalam tahap ini kita akan menghitung cosine_score yang diapatkan dari TF-IDF yang telah dilakukan pada tahap feature extraction sebelumnya. Hasil skor tersebut akan kita simpan dalam variabel "kemiripan" yang nantinya akan kita pakai untuk mendapatkan _cosine skor_ dari inputan user yang nantinya digunakan untuk  menampilkan rekomendasi film yang serupa

In [15]:
kemiripan = cosine_similarity(fitur_vectors)

In [16]:
kemiripan

array([[1.        , 0.02612462, 0.0191406 , ..., 0.004376  , 0.01810645,
        0.        ],
       [0.02612462, 1.        , 0.02037954, ..., 0.01899887, 0.00960984,
        0.        ],
       [0.0191406 , 0.02037954, 1.        , ..., 0.00903971, 0.03782564,
        0.00290443],
       ...,
       [0.004376  , 0.01899887, 0.00903971, ..., 1.        , 0.00828521,
        0.01183275],
       [0.01810645, 0.00960984, 0.03782564, ..., 0.00828521, 1.        ,
        0.00490778],
       [0.        , 0.        , 0.00290443, ..., 0.01183275, 0.00490778,
        1.        ]])

# Membuat sistem pengambilan judul dari inputan user ( based judul)

Pada tahap ini, kita akan membuat rancangan sistem step by step. Sistem ini akan melakukan pencocokan inputan judul film dari user dengan konten_film yang telah kita buat diatas sebelumnya yang sudah kita lakukan preprocesing menjadi hasil_normalisasi_corpus dan mendapatkan cosine skor

In [17]:
#membuat inputan
judul_film = input("Masukan Judul Film yang anda cari : ")

Masukan Judul Film yang anda cari : iron man


Lalu kita menguraikan data yang ada pada kolom "title" menjadi list, agar nantinya jika terjadi kesalahan penulisan judul , program dapat melakukan pembenaran penulisan judul dengan cara mencari penulisan judul yang terdekat dengan inputan user

In [18]:
#membuat kolom judul fuilm, menjadi list
list_judul_film_dari_dataset = data_film["title"].tolist()
print(list_judul_film_dari_dataset)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

Bisa kita lihat, pada inputan user terdapat kesalahan penulisan judul film, yang seharusnya 'Resident Evil" pada inputan adalah "residnt evil". Ini adalah fungsin dari library _difflib_. _difflib_ melakukan pencarian terdekat dari kesalahan input user terhadap data yang ada pada kolom "title". Guna ini untuk menghindari kesalahan yang mengakitbatkan data tidak ditemukan didalam database

In [19]:
# mencari judul film yang paling mirip dengan inputan dari user
pencarian_judul_terdekat_dari_user = difflib.get_close_matches(judul_film, list_judul_film_dari_dataset)
print(pencarian_judul_terdekat_dari_user)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


Setelah melakukan croscek terhadap judul yang diinput dengan judul yang ada di dalam kolom , kita mengambil judul film tersebut dan index dari film tersbut yang nantinya digunakan untuk urutan cosine skor

In [20]:
#Mengambil judul film dalam list yang paling mirip dengaqn inputan user
judul_paling_mirip = pencarian_judul_terdekat_dari_user[0]
judul_paling_mirip

'Iron Man'

In [21]:
#mencari index dari judul_paling_mirip
index_dari_judul_film = data_film[data_film.title==judul_paling_mirip]['index'].values[0]
print(index_dari_judul_film)

68


In [22]:
# mendaptkan list skor kermiripan dari index judul film yang dicari

kemiripan_skor = list(enumerate(kemiripan[index_dari_judul_film]))
print(kemiripan_skor)

[(0, 0.04433835516277472), (1, 0.03330846038529357), (2, 0.007888701817374113), (3, 0.002717426076620809), (4, 0.017383936937214402), (5, 0.007360671774422412), (6, 0.014934726027379525), (7, 0.1748282755020195), (8, 0.004805483888569053), (9, 0.035584646137753334), (10, 0.03662454308629372), (11, 0.006597141917537035), (12, 0.008889741496869013), (13, 0.005795212224269286), (14, 0.06123121813121173), (15, 0.003406012480153509), (16, 0.12425881969657074), (17, 0.01091351340238103), (18, 0.011435794866587443), (19, 0.005149564377933403), (20, 0.03323696402640668), (21, 0.014129776917382049), (22, 0.00413825859454311), (23, 0.008421396177277328), (24, 0.007821631177467571), (25, 0.0), (26, 0.11390867314008245), (27, 0.01485484534910772), (28, 0.03274328770482468), (29, 0.006083430276124395), (30, 0.059787544865853706), (31, 0.26851299084062363), (32, 0.003179625429165729), (33, 0.07844295170607006), (34, 0.0), (35, 0.02169627184047033), (36, 0.01604925361090819), (37, 0.00435110419849627

In [23]:
len(kemiripan_skor)

4803

In [24]:
# mengurutkan film berdasarkan skor kemiripan dari yang dicari user
urutan_kemiripan_film = sorted(kemiripan_skor, key = lambda x:x[1], reverse = True) 
print(urutan_kemiripan_film)

[(68, 1.0000000000000002), (79, 0.32393991275171347), (31, 0.26851299084062363), (7, 0.1748282755020195), (16, 0.12425881969657074), (511, 0.11895518618857498), (4401, 0.11867176172197863), (26, 0.11390867314008245), (3623, 0.10519130240758238), (64, 0.09872529542365954), (182, 0.09258595972769584), (174, 0.08676336982417157), (783, 0.08634830524281246), (3466, 0.0832376382890711), (1740, 0.0822664189266968), (4033, 0.08196862454527801), (203, 0.0807113892919014), (101, 0.079430346694805), (33, 0.07844295170607006), (46, 0.078145930304659), (1177, 0.07662832003401071), (1720, 0.07616129031323897), (1135, 0.07519870591616776), (138, 0.0745424463494311), (2229, 0.07424534302723994), (85, 0.07119080333382272), (198, 0.06929442820533022), (2235, 0.06878920342459217), (4325, 0.06809437288985337), (2875, 0.06801830996646739), (76, 0.06793484617740042), (122, 0.06777677133973882), (782, 0.06774145969507128), (421, 0.06762230162592477), (2487, 0.06751739665810591), (800, 0.06649307114964574), 

In [25]:
# pmenceteak rekomendasi film
print('Film yang anda cari : \n')

i = 1

for film in urutan_kemiripan_film:
  index = film[0]
  judul_dari_index = data_film[data_film.index==index]['title'].values[0]
  if (i<6):
    print(i, '.',judul_dari_index)
    i+=1

Film yang anda cari : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers


# Sistem Rekomendasi ( pengembangan)

Setelah sudah membuat ranbcangan sistemnya, kita sempurnakan dalam  pengembangan sistemnya. Pada sistem ini ditambahkan tampilan untuk pembenaran jika user melakukan tipografi

In [26]:
onoff = True

while onoff==True:
    print(" ")
    judul_film = input("Masukan Film yang anda cari  : ")
    judul_film = judul_film.lower()
    
    if judul_film=="off":
        print("Program selesai")
        sys.exit()

    list_judul_film_dari_dataset = data_film["title"].tolist()

    pencarian_judul_terdekat_dari_user = difflib.get_close_matches(judul_film, list_judul_film_dari_dataset)

    pencarian_judul_terdekat_dari_user_str = "".join(pencarian_judul_terdekat_dari_user[0])

    pencarian_judul_terdekat_dari_user_str = pencarian_judul_terdekat_dari_user_str.lower()
    
    if not pencarian_judul_terdekat_dari_user:
        print("Tidak ada judul film yang cocok ditemukan untuk "+judul_film)

    if judul_film!=pencarian_judul_terdekat_dari_user_str:
        yesNo = input("Maksud anda film : "+pencarian_judul_terdekat_dari_user_str+"? (yes/no)")
        yesNo = yesNo.lower()
        if yesNo=="yes":
            judul_paling_mirip = pencarian_judul_terdekat_dari_user[0]
            index_dari_judul_film = data_film[data_film.title==judul_paling_mirip]['index'].values[0]
            kemiripan_skor = list(enumerate(kemiripan[index_dari_judul_film]))
            urutan_kemiripan_film = sorted(kemiripan_skor, key = lambda x:x[1], reverse = True) 
            #masuk penampilan perulangan film
            i = 1
            for film in urutan_kemiripan_film:
              index = film[0]
              judul_dari_index = data_film[data_film.index==index]['title'].values[0]
              if (i<7):
                if(i==1):
                    print('Film anda ditemukan : ',judul_dari_index)
                    i+=1
                    print("=====================================")
                    print("5 rekomendasi film yang serupa dengan "+judul_dari_index)
                else:
                    print(i-1, '.',judul_dari_index)
                    i+=1
        else:
            print("Tidak ada judul film yang cocok ditemukan untuk "+judul_film)
    else:
        judul_paling_mirip = pencarian_judul_terdekat_dari_user[0]
        index_dari_judul_film = data_film[data_film.title==judul_paling_mirip]['index'].values[0]
        kemiripan_skor = list(enumerate(kemiripan[index_dari_judul_film]))
        urutan_kemiripan_film = sorted(kemiripan_skor, key = lambda x:x[1], reverse = True) 
        #masuk penampilan perulangan film
        i = 1
        for film in urutan_kemiripan_film:
          index = film[0]
          judul_dari_index = data_film[data_film.index==index]['title'].values[0]
          if (i<7):
            if(i==1):
                print('Film anda ditemukan : ',judul_dari_index)
                i+=1
                print("=====================================")
                print("5 rekomendasi film yang serupa dengan "+judul_dari_index)
            else:
                print(i-1, '.',judul_dari_index)
                i+=1
        

 
Masukan Film yang anda cari  : iron
Maksud anda film : nixon? (yes/no)yes
Film anda ditemukan :  Nixon
5 rekomendasi film yang serupa dengan Nixon
1 . Frost/Nixon
2 . Dick
3 . Primary Colors
4 . Swing Vote
5 . Man of the Year
 
Masukan Film yang anda cari  : off
Program selesai


SystemExit: 

STKI PROJEK 4 SELESAI

# Babak Tambahan : Membuat filter

In [27]:
data_film2 = pd.read_csv("Dataset/movies.csv")
data_film2

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honey

In [31]:
data_film2.fillna(" ",axis=0,inplace=True)
multi_genre_movies = data_film2[data_film2['genres'].str.contains('Action|Fiction', regex=True)]

multi_genre_movies

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4772,4772,31192,Drama Action Comedy,http://downterrace.blogspot.com/,42151,murder dark comedy crime family,en,Down Terrace,After serving jail time for a mysterious crime...,1.330379,[],"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2009-09-01,10000,89.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,You're only as good as the people you know.,Down Terrace,6.3,26,Robert Hill Robin Hill Julia Deakin David Scha...,"[{'name': 'Ben Wheatley', 'gender': 2, 'depart...",Ben Wheatley
4778,4778,0,Action Drama Crime Thriller,,38780,rampage massacre killing spree,en,Rampage,The boredom of small town life is eating Bill ...,7

In [41]:
genre_judul_vote = data_film2[["genres","title","vote_average"]]
genres = genre_judul_vote[genre_judul_vote['genres'].str.contains('Horror', regex=True)]
genres = genres.sort_values(by="vote_average",ascending=False)
genres.reset_index(drop="index")

,genres,title,vote_average
0,Drama Horror Thriller,Psycho,8.2
1,Horror Thriller,The Shining,8.1
2,Horror Action Thriller Science Fiction,Alien,7.9
3,Horror Mystery Science Fiction,The Thing,7.8
4,Horror Action Thriller Science Fiction,Aliens,7.7
...,...,...,...
514,Horror,American Beast,0.0
515,Thriller Horror,House at the End of the Drive,0.0
516,Horror,Her Cry: La Llorona Investigation,0.0
517,Horror,Midnight Cabaret,0.0


In [45]:
# Memisahkan data genre menjadi daftar kata-kata
genre_list = data_film2['genres'].apply(lambda x: x.split())

# Menggabungkan semua daftar kata-kata menjadi satu daftar genre
all_genres = [genre for sublist in genre_list for genre in sublist]

# Menghapus genre yang sama dan membuat daftar genre yang unik
unique_genres = list(set(all_genres))

# Menampilkan genre yang unik
print("Genre-genre yang termuat dalam data:")
for genre in unique_genres:
    print(genre)

Genre-genre yang termuat dalam data:
War
Action
Mystery
TV
Comedy
Fantasy
Movie
Family
Horror
Foreign
Music
Thriller
Documentary
Drama
Western
Adventure
Fiction
History
Science
Animation
Crime
Romance


In [47]:
genre_judul_vote = data_film2[["genres","title","vote_average"]]
genres = genre_judul_vote[genre_judul_vote['genres'].str.contains('Horror', regex=True)]
genres = genres.sort_values(by="vote_average",ascending=False)
genres.reset_index(drop="index")

,genres,title,vote_average
0,Drama Horror Thriller,Psycho,8.2
1,Horror Thriller,The Shining,8.1
2,Horror Action Thriller Science Fiction,Alien,7.9
3,Horror Mystery Science Fiction,The Thing,7.8
4,Horror Action Thriller Science Fiction,Aliens,7.7
...,...,...,...
514,Horror,American Beast,0.0
515,Thriller Horror,House at the End of the Drive,0.0
516,Horror,Her Cry: La Llorona Investigation,0.0
517,Horror,Midnight Cabaret,0.0


In [55]:
input_genre=["Horror"]
condition = data_film['genres'].apply(lambda x: any(genre in x for genre in input_genre))
genres = data_film[condition]
genres = genres.sort_values(by="vote_count", ascending=False)
genres[["genres","title","vote_count"]]
genres["title"].values[0]

'World War Z'